# Exploring Co-Morbidities

## Read data files

In [7]:
import os
import pandas as pd


datamatrixDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
icdDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240119/additional_data/2023-03-20-blood-pos-ICDs.csv')
admissionsDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240119/2023-11-10-blood-admissions.csv')

/tmp/ipykernel_663572/3850446445.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  icdDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240119/additional_data/2023-03-20-blood-pos-ICDs.csv')


In [8]:
icdDf.shape, datamatrixDf.shape, admissionsDf.shape

((456648, 7), (116754, 194), (436214, 20))

## Filter patients from the cohort

In [66]:
filteredIcdDf = datamatrixDf[['person_id', 'visit_occurrence_id']].drop_duplicates().merge(
    admissionsDf[['PATIENT_ID', 'EPISODE_ID', 'start_date']],
    how='inner',
    left_on=['person_id', 'visit_occurrence_id'],
    right_on=['PATIENT_ID', 'EPISODE_ID']
).drop(
    columns=['person_id', 'visit_occurrence_id']
).merge(
    admissionsDf[['PATIENT_ID', 'EPISODE_ID', 'start_date']].add_suffix('_PREV'),
    how='inner',
    left_on=['PATIENT_ID'],
    right_on=['PATIENT_ID_PREV']
).drop(
    columns=['PATIENT_ID', 'EPISODE_ID']
).merge(
    icdDf,
    how='inner',
    left_on=['PATIENT_ID_PREV', 'EPISODE_ID_PREV'],
    right_on=['PATIENT_ID', 'EPISODE_ID'],
).drop(
    columns=['PATIENT_ID', 'EPISODE_ID']
).rename(
    columns = {'PATIENT_ID_PREV': 'PATIENT_ID', 'EPISODE_ID_PREV': 'EPISODE_ID'}
)
filteredIcdDf = filteredIcdDf[((pd.to_datetime(filteredIcdDf.start_date) - pd.Timedelta(weeks=52)) < pd.to_datetime(filteredIcdDf.start_date_PREV)) & (pd.to_datetime(filteredIcdDf.start_date) >= pd.to_datetime(filteredIcdDf.start_date_PREV))]
filteredIcdDf = filteredIcdDf.drop(columns=['start_date', 'start_date_PREV'])
# filteredIcdDf = icdDf.merge(
#     datamatrixDf[['person_id', 'visit_occurrence_id']].drop_duplicates(),
#     how='inner',
#     left_on=['PATIENT_ID', 'EPISODE_ID'],
#     right_on=['person_id', 'visit_occurrence_id']
#     )
# icdDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape, filteredIcdDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape, datamatrixDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape
# (pd.to_datetime(filteredIcdDf.start_date) - pd.to_datetime(filteredIcdDf.start_date_PREV)).dt.days.hist(bins=100)
# filteredIcdDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape, filteredIcdDf[['PATIENT_ID_PREV', 'EPISODE_ID_PREV']].drop_duplicates().shape, datamatrixDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape
# filteredIcdDf[['PATIENT_ID']].drop_duplicates().shape, filteredIcdDf[['PATIENT_ID_PREV']].drop_duplicates().shape, datamatrixDf[['person_id']].drop_duplicates().shape
filteredIcdDf


,PATIENT_ID,EPISODE_ID,AUSTRALIAN_MODIFICATION,PREFIX,ICD_DESCRIPTION,RANK,ICD10CODE
24,2152832,583214,Z466,P,Fitting and adjustment of urinary device,1,NaN
25,2152832,583214,Z466,P,Fitting and adjustment of urinary device,1,NaN
26,2152832,583214,Z466,P,Fitting and adjustment of urinary device,1,NaN
27,2152832,583214,Z466,P,Fitting and adjustment of urinary device,1,NaN
28,2152832,583214,Z466,P,Fitting and adjustment of urinary device,1,NaN
...,...,...,...,...,...,...,...
26691590,1013298,15685286,R634,P,Abnormal weight loss,7,NaN
26691591,1013298,15685286,U823,P,Hypertension,10,NaN
26691592,1013298,15685286,R410,P,"Disorientation, unspecified",6,NaN
26691593,1013298,15685286,Z8643,P,Personal history of tobacco use disorder,9,Z86.4


## Obtaining Co-morbidities from Charlson Comorbidity Index

### Myocardial infarction

In [67]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I252'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                           
I214                     Acute subendocardial myocardial infarction                    4565
I211                     Acute transmural myocardial infarction of inferior wall       1447
I219                     Acute myocardial infarction, unspecified                       409
I210                     Acute transmural myocardial infarction of anterior wall        144
I213                     Acute transmural myocardial infarction of unspecified site      48
I252                     Old myocardial infarction                                       40
I212                     Acute transmural myocardial infarction of other sites           14
dtype: int64

In [71]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I252'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

5.69

### Congestive heart failure

In [72]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I099'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I110'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I130'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I132'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I255'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I420'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I425'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I426'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I427'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I428'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I429'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I43'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I50'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('P290'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                              
I500                     Congestive heart failure                                                                         12569
I509                     Heart failure, unspecified                                                                        9180
I501                     Left ventricular failure                                                                          7752
I420                     Dilated cardiomyopathy                                                                             869
I255                     Ischaemic cardiomyopathy                                                                           802
I429                     Cardiomyopathy, unspecified                                                                        600
I438                     Cardiomyopathy in other diseases classified elsewhere                                   

In [74]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I099'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I110'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I130'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I132'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I255'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I420'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I425'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I426'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I427'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I428'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I429'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I43'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I50'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('P290'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

12.58

## Peripheral vascular disease

In [75]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I70'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I71'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I731'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I738'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I739'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I771'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I790'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I792'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K551'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K558'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K559'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z958'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z959'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                          
I7021                    Atherosclerosis of arteries of extremities with intermittent claudication    527
I7020                    Atherosclerosis of arteries of extremities, unspecified                      441
Z958                     Presence of other cardiac and vascular implants and grafts                   430
I7023                    Atherosclerosis of arteries of extremities with ulceration                   428
I7022                    Atherosclerosis of arteries of extremities with rest pain                    233
I7101                    Dissection of thoracic aorta                                                 162
I7024                    Atherosclerosis of arteries of extremities with gangrene                     158
I714                     Abdominal aortic aneurysm, without mention of rupture                        132
K559                     Vascular disorder of intesti

In [76]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I70'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I71'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I731'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I738'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I739'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I771'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I790'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I792'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K551'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K558'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K559'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z958'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z959'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

2.53

## Cerebrovascular disease

In [77]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G45'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G46'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I6'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('H340'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                     
I678                     Other specified cerebrovascular diseases                                                2725
I611                     Intracerebral haemorrhage in hemisphere, cortical                                       2150
I634                     Cerebral infarction due to embolism of cerebral arteries                                1089
I639                     Cerebral infarction, unspecified                                                         961
I660                     Occlusion and stenosis of middle cerebral artery                                         604
G460                     Middle cerebral artery syndrome (I66.0+)                                                 568
I615                     Intracerebral haemorrhage, intraventricular                                              390
I601                     Subarachnoid haemorrhage from middle ce

In [78]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G45'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G46'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I6'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('H340'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

5.49

### Dementia

In [79]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F00'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F01'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F02'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F03'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G30'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F051'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G311'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                            
F051                     Delirium superimposed on dementia                                                              744
F009                     Dementia in Alzheimer's disease, unspecified (G30.9+)                                          199
G309                     Alzheimer's disease, unspecified                                                               195
F019                     Vascular dementia, unspecified                                                                 119
F03                      Unspecified dementia                                                                           108
F023                     Dementia in Parkinson's disease (G20+)                                                          67
G308                     Other Alzheimer's disease                                                                       61
F002           

In [80]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F00'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F01'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F02'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F03'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G30'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('F051'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G311'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

2.56

## Chronic pulmonary disease

In [81]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I278'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I279'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J40')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J41')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J42')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J43')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J44')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J45')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J46')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J47'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J60')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J61')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J62')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J63')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J64')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J65')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J66')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J67'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J684'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J701'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J703'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                             
J440                     Chronic obstructive pulmonary disease with acute lower respiratory infection    6196
J441                     Chronic obstructive pulmonary disease with acute exacerbation, unspecified      2835
J459                     Asthma, unspecified                                                             1627
J47                      Bronchiectasis                                                                  1301
J449                     Chronic obstructive pulmonary disease, unspecified                               389
J450                     Predominantly allergic asthma                                                    192
J448                     Other specified chronic obstructive pulmonary disease                            145
J439                     Emphysema, unspecified                                                           125
J40               

In [82]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I278'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I279'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J40')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J41')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J42')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J43')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J44')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J45')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J46')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J47'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J60')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J61')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J62')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J63')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J64')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J65')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J66')) | (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J67'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J684'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J701'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('J703'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

5.42

## Rheumatologic disease

In [83]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M05'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M06'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M315'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M32'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M33'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M34'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M351'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M353'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M360'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                               
M349                     Systemic sclerosis, unspecified                                   228
M0699                    Rheumatoid arthritis, unspecified, site unspecified               111
M321                     Systemic lupus erythematosus with organ or system involvement      69
M348                     Other forms of systemic sclerosis                                  56
M0694                    Rheumatoid arthritis, unspecified, hand                            52
M0696                    Rheumatoid arthritis, unspecified, lower leg                       39
M353                     Polymyalgia rheumatica                                             18
M0698                    Rheumatoid arthritis, unspecified, other site                      18
M328                     Other forms of systemic lupus erythematosus                        16
M0695                    Rheumatoid arthritis, unspecifie

In [84]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M05'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M06'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M315'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M32'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M33'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M34'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M351'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M353'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('M360'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

1.01

## Peptic ulcer

In [85]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K25'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K26'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K27'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K28'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K29'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                    
K2970                    Gastritis, unspecified, without mention of haemorrhage                                 757
K2960                    Other gastritis, without mention of haemorrhage                                        489
K2971                    Gastritis, unspecified, with haemorrhage                                               298
K291                     Other acute gastritis                                                                  286
K2950                    Chronic gastritis, unspecified, without mention of haemorrhage                         212
K254                     Gastric ulcer, chronic or unspecified with haemorrhage                                 202
K2980                    Duodenitis, without mention of haemorrhage                                             162
K264                     Duodenal ulcer, chronic or unspecified with haemorrhag

In [86]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K25'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K26'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K27'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K28'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K29'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

3.14

## Hemiplegia/paraplegia

In [87]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G041'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G114'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G801'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G802'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G81'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G82'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G830'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G831'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G832'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G833'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G834'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G839'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                              
G819                     Hemiplegia, unspecified                          2269
G834                     Cauda equina syndrome                             119
G832                     Monoplegia of upper limb                           66
G8242                    Spastic tetraplegia, unspecified, chronic          48
G8225                    Paraplegia, unspecified, incomplete, acute         26
G833                     Monoplegia, unspecified                            19
G8205                    Flaccid paraplegia, incomplete, acute              17
G8250                    Tetraplegia, unspecified, unspecified              10
G8200                    Flaccid paraplegia, unspecified                     9
G8252                    Tetraplegia, unspecified, chronic                   8
G831                     Monoplegia of lower limb                            6
G839                     Paralytic syndrome, unspecified    

In [88]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G041'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G114'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G801'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G802'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G81'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G82'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G830'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G831'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G832'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G833'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G834'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('G839'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

2.35

## Diabetes without complications

In [89]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E100'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E101'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E106'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E108'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E109'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E110'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E111'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E116'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E118'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E119'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E120'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E121'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E126'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E128'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E129'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E130'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E131'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E136'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E138'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E139'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E140'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E141'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E146'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E148'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                                                    
E119                     Type 2 diabetes mellitus without complication                                                                          70774
E139                     Other specified diabetes mellitus without complication                                                                 33070
E1165                    Type 2 diabetes mellitus with poor control                                                                             16172
E1164                    Type 2 diabetes mellitus with hypoglycaemia                                                                            11530
E1365                    Other specified diabetes mellitus with poor control                                                                     5286
E1364                    Other specified diabetes mellitus with hypoglycaemia                                

In [90]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E100'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E101'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E106'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E108'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E109'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E110'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E111'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E116'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E118'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E119'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E120'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E121'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E126'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E128'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E129'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E130'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E131'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E136'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E138'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E139'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E140'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E141'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E146'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E148'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

14.81

## Diabetes with chronic complications

In [91]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E102'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E103'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E104'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E105'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E107'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E112'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E115'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E117'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E122'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E123'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E124'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E125'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E127'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E132'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E133'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E134'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E135'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E137'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E142'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E143'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E144'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E145'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E147'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                                            
E1122                    Type 2 diabetes mellitus with established diabetic nephropathy                                                 27579
E1172                    Type 2 diabetes mellitus with features of insulin resistance                                                   23437
E1171                    Type 2 diabetes mellitus with multiple microvascular and other specified nonvascular complications             21125
E1129                    Type 2 diabetes mellitus with other specified kidney complication                                              20738
E1151                    Type 2 diabetes mellitus with peripheral angiopathy, without gangrene                                          11417
E1121                    Type 2 diabetes mellitus with incipient diabetic nephropathy                                                    4737
E1173          

In [92]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E102'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E103'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E104'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E105'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E107'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E112'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E115'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E117'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E122'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E123'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E124'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E125'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E127'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E132'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E133'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E134'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E135'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E137'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E142'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E143'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E144'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E145'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('E147'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

14.3

## Mild liver disease

In [93]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B18'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K700'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K701'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K702'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K703'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K709'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K713'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K714'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K715'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K717'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K73'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K74'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K760'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K762'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K763'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K764'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K768'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K769'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z944'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                  
B181                     Chronic viral hepatitis B without delta-agent        18893
B182                     Chronic viral hepatitis C                            18026
K746                     Other and unspecified cirrhosis of liver              3774
K703                     Alcoholic cirrhosis of liver                          3476
K760                     Fatty (change of) liver, not elsewhere classified     3338
K769                     Liver disease, unspecified                            2893
K768                     Other specified diseases of liver                     2436
K701                     Alcoholic hepatitis                                    269
K763                     Infarction of liver                                    157
Z944                     Liver transplant status                                112
K709                     Alcoholic liver disease, unspecified                    76
K

In [94]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B18'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K700'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K701'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K702'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K703'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K709'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K713'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K714'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K715'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K717'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K73'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K74'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K760'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K762'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K763'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K764'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K768'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K769'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z944'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

7.26

## Moderate/severe liver disease

In [95]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I850'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I859'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I864'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I982'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K704'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K711'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K721'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K729'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K765'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K766'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K767'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                 
K765                     Hepatic veno-occlusive disease                                                      1947
K729                     Hepatic failure, unspecified                                                        1446
K704                     Alcoholic hepatic failure                                                            785
I982                     Oesophageal varices without mention of bleeding in diseases classified elsewhere     744
K767                     Hepatorenal syndrome                                                                 356
I864                     Gastric varices                                                                      329
K766                     Portal hypertension                                                                  245
I859                     Oesophageal varices without bleeding                                   

In [96]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I850'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I859'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I864'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I982'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K704'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K711'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K721'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K729'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K765'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K766'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('K767'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

2.18

## Renal disease

In [97]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I120'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I131'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N032'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N033'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N034'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N035'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N036'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N037'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N052'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N053'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N054'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N055'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N056'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N057'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N18'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N19'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N250'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z490'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z491'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z492'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z940'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z992'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                             
Z491                     Extracorporeal dialysis                                                         224713
N185                     Chronic kidney disease, stage 5                                                  15104
N183                     Chronic kidney disease, stage 3                                                  10243
Z940                     Kidney transplant status                                                          6703
N19                      Unspecified kidney failure                                                        6115
N182                     Chronic kidney disease, stage 2                                                   4599
I120                     Hypertensive kidney disease with kidney failure                                   3797
N184                     Chronic kidney disease, stage 4                                                   3497
Z4

In [98]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I120'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('I131'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N032'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N033'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N034'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N035'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N036'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N037'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N052'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N053'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N054'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N055'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N056'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N057'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N18'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N19'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('N250'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z490'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z491'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z492'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z940'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('Z992'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

12.73

## Any malignancy (tumor, leukemia, lymphoma)

In [99]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C0'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C1'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C20'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C23'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C24'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C25'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C26'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C30'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C31'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C32'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C33'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C34'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C37'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C38'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C39'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C40'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C41'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C43'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C45'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C46'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C47'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C48'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C49'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C50'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C51'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C52'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C53'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C54'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C55'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C56'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C57'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C58'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C6'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C70'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C71'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C72'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C73'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C74'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C75'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C76'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C81'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C82'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C83'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C84'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C85'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C88'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C90'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C91'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C92'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C93'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C94'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C95'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C96'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C97'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                  
C9200                    Acute myeloblastic leukaemia [AML], without mention of remission                     472110
C9201                    Acute myeloblastic leukaemia [AML], in remission                                     246697
C9100                    Acute lymphoblastic leukaemia [ALL], without mention of remission                    137199
C9101                    Acute lymphoblastic leukaemia [ALL], in remission                                    131966
C9280                    Acute myeloid leukaemia with multilineage dysplasia, without mention of remission     92880
                                                                                                               ...  
C340                     Malignant neoplasm of main bronchus                                                       1
C502                     Malignant neoplasm of upper-inner quadrant of bre

In [100]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C0'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C1'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C20'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C23'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C24'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C25'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C26'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C30'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C31'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C32'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C33'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C34'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C37'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C38'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C39'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C40'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C41'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C43'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C45'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C46'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C47'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C48'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C49'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C50'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C51'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C52'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C53'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C54'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C55'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C56'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C57'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C58'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C6'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C70'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C71'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C72'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C73'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C74'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C75'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C76'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C81'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C82'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C83'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C84'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C85'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C88'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C90'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C91'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C92'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C93'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C94'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C95'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C96'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C97'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

16.27

## Metastatic solid tumor

In [101]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C77'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C78'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C79'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C80'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                    
C786                     Secondary malignant neoplasm of retroperitoneum and peritoneum                         5008
C771                     Secondary and unspecified malignant neoplasm of intrathoracic lymph nodes              3707
C795                     Secondary malignant neoplasm of bone and bone marrow                                   3617
C782                     Secondary malignant neoplasm of pleura                                                 3572
C787                     Secondary malignant neoplasm of liver and intrahepatic bile duct                       2865
C772                     Secondary and unspecified malignant neoplasm of intra-abdominal lymph nodes            2537
C780                     Secondary malignant neoplasm of lung                                                   2261
C773                     Secondary and unspecified malignant neoplasm of

In [102]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C77'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C78'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C79'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('C80'))

].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

3.24

## HIV/AIDS

In [103]:
filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B20'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B24'))
][['AUSTRALIAN_MODIFICATION', 'ICD_DESCRIPTION']].value_counts()

AUSTRALIAN_MODIFICATION  ICD_DESCRIPTION                                                                          
B24                      Unspecified human immunodeficiency virus [HIV] disease                                       908
B21                      Human immunodeficiency virus [HIV] disease resulting in malignant neoplasms                  890
B20                      Human immunodeficiency virus [HIV] disease resulting in infectious and parasitic diseases    105
dtype: int64

In [104]:
round(filteredIcdDf[
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B20'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B21'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B22'))
    |
    (filteredIcdDf.AUSTRALIAN_MODIFICATION.str.startswith('B24'))
].PATIENT_ID.unique().shape[0]/datamatrixDf.person_id.unique().shape[0] * 100, 2)

0.76